# SQL

### Introducción

Durante esta actividad vamos a aprender los conceptos básicos de SQL. Vamos a aprender a insertar, eliminar y actualizar datos de una base de datos además de hacer consultas sencillas sobre ellos.

### Requisitos

Para esta actividad, así como en las siguientes actividades de SQL vamos a utilizar

- Python 3
- Jupyter
- La librería `ipython-sql`. Para instalarla pueden hacerlo con `pip3`:
```
pip3 install ipython-sql
```
- SQLite3, que viene junto con la instalación de Python

### Outline

En esta actividad aprendermos a:

- Crear y modificar tablas.
- Insertar, eliminar y actualizar datos.
- Crear llaves en las tablas.
- Hacer consultas que involucren selección, proyección, producto cruz y operaciones de conjuntos.
- Consultas básicas de agregación.

### Esquema

Para esta actividad vamos a trabajar con el siguiente esquema:

- `Capitanes(cid INT PRIMARY KEY, cnombre VARCHAR(100), crating FLOAT, cedad INT)`
- `Botes(bid INT PRIMARY KEY, bnombre VARCHAR(100), bcolor VARCHAR(100))`
- `Reservas(cid INT, bid INT, fecha DATE, PRIMARY KEY(cid, bid))`

Que corresponde a capitanes que reservan ciertos botes. El `cid` y `bid` en la tabla `Reservas` proviene de las tablas `Capitanes` y `Botes` respectivamente.

## Actividad

### Crear tablas, eliminar tablas y llaves primarias

En esta actividad vamos a utilizar `ipython-sql` para tener acceso a SQL desde este _notebook_. Lo vamos a importar a continuación.

In [ ]:
%load_ext sql

Ahora vamos a crear una nueva base de datos para esta actividad. Esta base de datos se llamará `capitanes.db`.

In [ ]:
%sql sqlite:///capitanes.db

**Ojo**: cada vez que anteponemos `%sql` es porque esa línea corresponde a un comando a SQL que va a la base de datos en la que estamos trabajando (en este caso `capitanes.db`). Si queremos tener todo un bloque con instrucciones SQL tenemos que usar `%%sql` (habrán ejemplos de esto a lo largo del _notebook_).

Tu primera prueba será crear una base de datos de acuerdo el esquema de arriba. La sintaxis para crear tablas en SQL es la siguiente (si están trabajando desde el cliente, después de cada sentencia SQL debes poner un punto y coma `;`):

```SQL
CREATE TABLE <Nombre Tabla> (<atributo_1> tipo, ... , <attributo_N> tipo)
```

Por ejemplo, para crear la tabla de capitanes:

In [ ]:
%%sql 
DROP TABLE IF EXISTS Capitanes;
CREATE TABLE Capitanes(cid INT, cnombre VARCHAR(100), crating FLOAT, cedad INT);

En la celda anterior estamos eliminando la tabla si es que existe para luego crear una tabla según lo requerido por el esquema. Pero cuidado! Olvidamos agregar la llave primaria, por lo que vamos a **eliminar** la tabla y crearla de nuevo. Además, vamos a agregar un valor por defecto al atributo `crating`, que en este caso será 0.

In [ ]:
%%sql 
DROP TABLE Capitanes;
CREATE TABLE Capitanes(cid INT PRIMARY KEY, cnombre VARCHAR(100), crating FLOAT DEFAULT 0, cedad INT);

Otra opción es agregar la llave primaria al final de la instrucción `CREATE TABLE`. Para ver un ejemplo, vamos a crear la tabla de reservas.

In [ ]:
%%sql 
DROP TABLE IF EXISTS Reservas;
CREATE TABLE Reservas(cid INT, bid INT, fecha DATE, PRIMARY KEY(cid, bid));

En el caso anterior, tenemos una llave primaria compuesta. En general, para crear una tabla con llave primaria, lo señalamos al final, de la forma:

```SQL
CREATE TABLE <Nombre Tabla> (
    <atributo_1> tipo, ..., 
    <attributo_N> tipo, 
    PRIMARY KEY(<atributos separados por coma (,)>))
```

**1.** Ahora es tu turno. En la celda a continuación debes crear la tabla de botes. No olvides agregar su llave primaria.

In [ ]:
%sql

### Tipos de datos

Al crear las tablas disponemos varios tipos de datos. Por ahora destacamos los siguientes:

- Caracteres (_Strings_):
  - `CHAR(20)`: _Strings_ de largo fijo.
  - `VARCHAR(20)`: _Strings_ de largo variable.
  
- Números:
  - `INT`
  - `FLOAT`
  - `SMALLINT`
  
- Tiempos y fechas:
  - `DATE`: fecha.
  - `TIME`: hora.
  - `TIMESTAMP`: fecha y hora.

### Insertar, eliminar y modificar elementos en tablas

Si ejecutamos la consulta `SELECT * FROM Capitanes` notaremos que el resultado es vacío. Esto es evidente, ya que esta consulta me retorna todo lo que tengo en la tabla `Capitanes` y actualmente no hemos insertado nada.

In [ ]:
%sql SELECT * FROM Capitanes;

Para insertar valores, la forma básica es la siguiente:

```SQL
INSERT INTO <Nombre Tabla> 
VALUES (<valor atributo 1> , ..., <valor atributo N>)
```

Por ejemplo si ejecutamos la consulta:

```SQL
INSERT INTO Capitanes 
VALUES(1, 'Claudio', 0, 35)
```

estamos insertando un Capitán con `cid` 1, `cnombre` Claudio, `crating` 0 y `cedad` 35.

In [ ]:
%sql INSERT INTO Capitanes VALUES(1, 'Claudio', 0, 35)

Recordemos que habíamos señalado que la llave primaria de la tabla `Capitanes` era el `cid`. Veamos que pasa si intentamos insertar un capitán con el mismo `cid`.

In [ ]:
%sql INSERT INTO Capitanes VALUES(1, 'Claudio Bravo', 0, 35)

La consulta anterior debería haber arrojado un error. Así que vamos a cambiar el `cid`. Sin embargo, vamos a omitir algunos atributos.

In [ ]:
%sql INSERT INTO Capitanes(cid, cnombre) VALUES(13, 'Claudio Bravo')

In [ ]:
%sql SELECT * FROM Capitanes

Si te das cuenta, ahora tenemos dos tuplas en la tabla de capitanes. Como podrías haber esperado, dado que omitimos el valor para ciertos atributos en la segunda inserción, el `crating` tomó el valor _default_. Pero ojo! el atributo `cedad` tomó un valor **nulo**. Más adelante en el curso vamos a ahondar en esta temática y ver por qué los valores nulos pueden ser un problema. Ahora vamos a agregar una nueva tupla para hacer algunas consultas sencillas.

In [ ]:
%sql INSERT INTO Capitanes VALUES(23, 'Arturo Vidal', 8, 31)

In [ ]:
%sql SELECT * FROM Capitanes;

In [ ]:
%sql SELECT * FROM Capitanes WHERE cnombre='Claudio'

En la consulta anterior estamos filtrando por Capitanes donde su nombre sea `'Claudio'`. Si queremos que el nombre **contenga** `'Claudio'` usamos la instrucción `LIKE`:

In [ ]:
%sql SELECT * FROM Capitanes WHERE cnombre LIKE '%Claudio%'

Pueden notar que hemos añadido un `%` al _string_. Esto es porque antes y después de `Claudio` queremos permitir cualquier secuencia de caracteres. Para el `LIKE` tenemos dos instrucciones posibles:

- `%` que significa cualquier secuencia de caracteres.
- `_` que significa un caracter, pero **solamente** uno.

Como no estamos seguros de mantener a `Claudio` en la tabla `Capitanes` lo vamos a eliminar. Para eliminar tuplas lo hacemos con la instrucción `DELETE`. Veamos que pasa al ejecutar la siguiente consulta:

In [ ]:
%%sql
DELETE FROM Capitanes
WHERE cnombre LIKE '%Claudio%'

In [ ]:
%sql SELECT * FROM Capitanes

Lo que hicimos fue borrar de la tabla `Capitanes` todas las tuplas que satisfacen la condición en el `WHERE` de la consulta. Para actualizar una tupla usamos la instrucción `UPDATE`, como veremos a continuación:

In [ ]:
%sql UPDATE Capitanes SET cnombre='King Arturo' WHERE cid=23

In [ ]:
%sql SELECT * FROM Capitanes

### Modificando tablas

En SQL es posible modificar tablas. Por ejemplo si quisieramos eliminar el atributo `crating` podríamos ejecutar la consulta:

```SQL
ALTER TABLE Capitanes DROP COLUMN crating
```

O agregar una columna, como por ejemplo:

```SQL
ALTER TABLE Capitanes ADD COLUMN rut VARCHAR(20)
```

### Consultas básicas en SQL

Para comenzar a hacer consultas primero debemos llenar nuestras tablas. Asegurate que las instancias de las tablas sean las siguientes:

#### Capitanes

| cid | cnombre     | crating | cedad |
|-----|-------------|---------|-------|
| 23  | King Arturo | 8       | 31    |
| 29  | Juan        | 1       | 33    |
| 31  | Andy        | 8       | 55    |
| 32  | Felipe      | 8.4     | 25    |
| 58  | Oscar       | 10      | 35    |
| 64  | Isidora     | 7.5     | 35    |
| 71  | Pedro       | 10      | 16    |
| 74  | Isidora     | 9       | 35    |
| 85  | Rosa        | 3       | 25    |
| 95  | Romano      | 5.5     | 63    |

#### Botes

| bid | bnombre   | bcolor |
|-----|-----------|--------|
| 101 | Catamaran | Azul   |
| 102 | Catamaran | Rojo   |
| 103 | Endurance | Verde  |
| 104 | Yate      | Rojo   |

#### Reservas

| cid | bid | fecha    |
|-----|-----|----------|
| 23  | 101 | 10/10/16 |
| 23  | 102 | 10/10/16 |
| 23  | 103 | 8/10/16  |
| 23  | 104 | 7/10/17  |
| 31  | 102 | 10/11/17 |
| 31  | 103 | 6/11/18  |
| 31  | 104 | 12/11/18 |
| 64  | 101 | 5/9/18   |
| 64  | 102 | 8/9/18   |
| 74  | 103 | 8/9/18   |

**Hint**: Para insertar un tipo fecha en SQLITE tienes que ingresarlo entre comillas simples en el formato `'YYYY-MM-DD'`, por ejemplo:

In [ ]:
%sql INSERT INTO Reservas VALUES(23, 101, '2018-10-10')

Las consultas más básicas son de la forma `SELECT - FROM - WHERE`. En general, la consulta de álgebra relacional:

$$
\pi_{a_1, \dots, a_n}(\sigma_{\text{condiciones}}(R_1 \times R_m))
$$

se traduce en SQL como:

```SQL
SELECT a_1, ..., a_n
FROM R_1, ..., R_m
WHERE <condiciones>
```

**2.** Prueba la siguiente consulta:

In [ ]:
%sql SELECT cid, cnombre, crating, cedad FROM Capitanes

¿Qué es lo que debería hacer la consulta anterior? Recuerda que si lo que quieres es proyectar todos los atributos puedes usar `*`.

**3.** Ahora realiza dos consultas, una para obtener todo desde la tabla `Reservas` y todo de la tabla `Botes`. Revisa que agregaste todos los datos correctamente.

In [ ]:
# Reserva
%sql

In [ ]:
# Botes
%sql

Ahora ejecuta la consulta a continuación. ¿Qué crees que retorna?

In [ ]:
%sql SELECT * FROM Capitanes WHERE cnombre='Juan'

Ahora es tu turno, realiza las siguientes consultas en SQL:

**3.2.** Busca los nombres de los capitanes con rating más de 4 (prueba seleccionando `*` y luego solo el atributo `cnombre`).

**Ayuda**: Puedes usar los filtros `x = y`, `x < y`, `x <= y`, `<>` (distinto), etc. Para los números es obvio, para los strings es orden lexicográfico.

In [ ]:
%sql

**4.** Similarmente, busca los nombres de los capitanes con rating más de 4 y cuya edad está entre 25 y 40 años.

**Ayuda**: Combina condiciones en el `WHERE` usando `AND` y `OR`. Recuerda que para negar algo puedes usar `NOT`.

In [ ]:
%sql

**5.** Busca la edad de los capitanes que no tienen una `'a'` en su nombre (recuerda usar `LIKE`).

In [ ]:
%sql

Escribe una consulta que entregue los nombres y rating de los capitanes, pero ordenado por `crating`. Para ordenar agrega un `ORDER BY (<atr_1>, ..., <atr_N>) DESC` al final de tu consulta. Eso ordena los resultados por el atributo `<atr_1>`, después `<atr_2>`, etc. Prueba que pasa al eliminar el `DESC` al final del `ORDER BY`.

### Consultas de _join_, unión, intersección y diferencia

Imaginemos que queremos encontrar los nombres de los capitanes que reservaron el bote 103. 

**6.** Escribe primero esta consulta en Álgebra relacional.

Una forma directa para traspasarlo a SQL es con el comando `INNER JOIN`:

In [ ]:
%%sql
SELECT  cnombre
FROM  Capitanes INNER JOIN Reservas
ON Capitanes.cid = Reservas.cid
WHERE Reservas.bid = 103

Este es un buen minuto para repasar los joins. Puedes correr la consulta de arriba sin el `WHERE` para ver lo que sería el resultado de $\text{Capitanes} \bowtie_{\text{cid}=\text{cid}} \text{Reservas}$. ¿Qué pasa al agregar la condición del `WHERE`?
Si bien existe la instrucción `INNER JOIN`, una forma más clara (o que a nosotros nos gusta más) es explotando el producto cartesiano:

In [ ]:
%%sql
SELECT  *
FROM  Capitanes, Reservas
WHERE Capitanes.cid = Reservas.cid
AND Reservas.bid = 103

En este caso estamos aplicando el `WHERE` al Producto Cartesiano, y por eso incluimos la condición `WHERE Capitanes.cid = Reservas.cid`. ¿Entiendes por qué ambas consultas son equivalentes?

Ahora intenta escribir las siguientes consultas:

**7.** Los nombres de los capitanes que reservaron un bote rojo.

In [ ]:
%sql

**8.** Los nombres de los capitanes que reservaron un bote rojo o un bote verde.

In [ ]:
%sql

**9.** La misma anterior, pero usando `UNION` para unir el resultado de dos consultas. Prueba usando `UNION` y luego `UNION ALL`. ¿Cuál es la diferencia?

In [ ]:
%sql

**10.** Usa `AS` para cambiar de nombres a los atributos (esto es útil no solo para la unión!). La siguiente consulta entrega todos los ids del sistema:

In [ ]:
%%sql
SELECT cid AS id
FROM  Capitanes
UNION
SELECT bid AS id
FROM Botes

**11.** Los nombres de los capitanes que reservaron un bote rojo y un bote verde. Puedes usar `INTERSECT` para intersectar el resultado de dos consultas. ¿Puedes escribir también esta consulta usando AND?

In [ ]:
%sql

### Sub-consultas y consultas anidadas

Has observado que cada consulta entrega una tabla, los atributos de esa tabla son lo que pones en el `SELECT`. Esto significa que podemos poner cualquier consulta SQL en el `FROM`. Vamos a ver cómo realizar esto.

**12.** Escribe una consulta que entregue los nombres de los capitanes junto a los bid de los botes que han reservado, pero contando solo las reservas hechas después del primero de octubre del 2017.

In [ ]:
%sql

Sea $Q$ la consulta anterior. ¿Qué va a retornar la siguiente consulta? (Ojo, tienes que reemplazar el valor de $Q$ en la celda siguiente)

In [ ]:
%%sql
SELECT Botes.bid, bnombre, bcolor
FROM Botes, (Q) AS Res2017
WHERE Botes.bid = Res2017.bid

**Importante**: cada vez que quieras anidar una consulta en el `FROM`, debes darle un alias. En el caso anterior el alias es Res2017.

**13.** Ahora modifica $Q$ para que solo retorne los bid de los botes, y no los nombres de los capitanes. ¿Qué retorna la consulta a continuación? ¿Qué pasa si Q entrega dos atributos en vez de uno (es decir, usando $Q$ sin modificar)? ¿Por qué pasa esto?

In [ ]:
%%sql
SELECT Botes.bid, bnombre, bcolor
FROM Botes
WHERE Botes.bid IN Q

También podemos usar consultas anidadas en el `WHERE`. Además del `IN` podemos usar `NOT IN`, o `> ANY`, `> ALL` para cualquier comparación entre `<`, `>`, `<=`, `>=`, `=`, y también anteponiendo `NOT`.

**Ojo**: `ANY` y `All` no funcionan en SQLite, pero es muy común en los otros sistemas.

Por ejemplo, si queremos encontrar los capitanes que no han reservado un bote verde:

1. Primero escribe una consulta que entregue los `cid` de los capitanes que han reservado un bote verde.
2. Sea $Q$ la consulta anterior. Los capitanes que no han reservado un bote verde se pueden extraer remplazando $Q$ en:

In [ ]:
%%sql
SELECT cnombre
FROM capitanes
WHERE cid NOT IN (Q)

Ahora:

**14.** Encuentre los nombres de los capitanes que tienen más de 20 años y que no han reservado un bote rojo. ¿Puedes escribir esto sin usar consultas anidadas?

In [ ]:
%sql

**15.** Encuentre los nombres de los capitanes que reservaron al menos dos botes.

In [ ]:
%sql

**16.** Encuentre los nombres de los capitanes que han reservado todos los botes. ¿Puedes escribir esto sin usar consultas anidadas?

In [ ]:
%sql

### Agregación

¿Qué pasa si queremos el capitán con el mejor rating? Primero, podemos obtener el mejor rating con la siguiente consulta:

In [ ]:
%%sql
SELECT MAX(crating)
FROM Capitanes

Luego seleccionamos los nombres de aquellos capitanes con ese rating:

In [ ]:
%%sql
SELECT cnombre
FROM Capitanes
WHERE crating = (SELECT MAX(crating)
                FROM Capitanes)

En general SQL soporta varias funciones de agregación: `MAX`, `MIN` (mínimo), `AVG` (promedio), `SUM` (suma). Intenta hacer las siguientes consultas:

**17.** Encuentra el rating promedio de los capitanes mayores a 45 años.

In [ ]:
%sql

**18.** Encuentra los capitanes cuya edad es menor que el promedio.

In [ ]:
%sql

**19.** Encuentra los botes que no han sido reservados por el capitán con peor rating.

In [ ]:
%sql

También podemos usar `COUNT` para contar tuplas. Hay dos versiones (pruébalas!):

- `SELECT COUNT(*) FROM Tabla` cuenta cuantas tuplas tiene la tabla.
- `SELECT COUNT(DISTINCT cnombre) FROM Capitanes` cuenta cuantos nombres distintos hay en Capitanes.

En general `SELECT COUNT(*) FROM Tabla` y S`ELECT COUNT(atributo) FROM Tabla` es lo mismo, pero hay casos en los que es distinto: cuando las tablas tienen valores nulos o indefinidos. Hablaremos sobre esto más adelante en el curso.

Ahora realiza las siguientes consultas.

**20.** Encuentra cuantos capitanes menores de 40 años hay.

In [ ]:
%sql

**21.** Cuenta la cantidad de reservas hechas a botes de color rojo.

In [ ]:
%sql

**22.** Cuenta cuantos capitanes tienen rating mayor al promedio y luego cuantos capitanes tienen rating menor al promedio.

In [ ]:
%sql

Finalmente podemos usar `GROUP BY` para ir agrupando los resultados. Prueba con la consulta:

In [ ]:
%%sql
SELECT cnombre, SUM(cedad)
FROM Capitanes
GROUP BY cnombre

¿Qué pasó con Isidora? Lo que hacemos es agrupar a las dos tuplas con nombre Isidora en una, y sumamos la edad. Prueba ahora:

In [ ]:
%%sql
SELECT bcolor, count(bcolor)
FROM Botes
GROUP BY bcolor

Ahora realiza las siguientes consultas:

**23.** Encuentra para los capitanes el promedio del ranking por cada edad distinta.

In [ ]:
%sql

**24.** Encuentra el número de reservas por cada color de bote.

In [ ]:
%sql

**25.** Encuentra el numero de reservas por capitán y por color de bote.

In [ ]:
%sql

### Inserción y eliminación más compleja

Lo último que veremos es una forma de insertar y eliminar tuplas de forma más compleja. Por ejemplo podemos insertar valores de una consulta.

**26.** Crea una tabla `CapitanesTop(cid INT, cnombre VARCHAR(100))`

In [ ]:
%sql

Ahora ejecuta la siguiente inserción. Nota que para insertar desde una consulta no debemos escribir la instrucción `VALUES`.

In [ ]:
%%sql
INSERT INTO CapitanesTop
(SELECT cid,cnombre
FROM Capitanes
WHERE Rating > 8)

Finalmente, si quieres dejar capitanes que tienen un rating igual o superior a 9 puedes escribir la siguiente consulta:

In [ ]:
%%sql
DELETE FROM CapitanesTop
WHERE cid IN
  (SELECT cid
  FROM Capitanes
  WHERE crating < 9)

### Agregación en SQL

Vamos a crear una nueva tabla para ejecutar consultas de agregación.

In [ ]:
%sql DROP TABLE IF EXISTS productos
%sql CREATE TABLE productos(nombre VARCHAR(20), precio INT, categoria VARCHAR(20), fabricante VARCHAR(20), año_fabricacion INT)

%sql INSERT INTO productos VALUES('Yogurt Frutilla 225ml', 690, 'Lácteo', 'Nestlé', 2018)
%sql INSERT INTO productos VALUES('Yogurt Frutilla 1L', 990, 'Lácteo', 'Nestlé', 2018)
%sql INSERT INTO productos VALUES('Manjar 500ml', 450, 'Lácteo', 'Nestlé', 2019)
%sql INSERT INTO productos VALUES('Manjar 500ml', 1290, 'Lácteo', 'Nestlé', 2019)
%sql INSERT INTO productos VALUES('Manjar 500ml', 1500, 'Lácteo', 'Colún', 2019)
%sql SELECT * FROM productos

¿Qué hace esta consulta?

In [ ]:
%%sql
SELECT AVG(precio)
FROM productos
WHERE fabricante = 'Nestlé'

También podemos usar `SUM`, `MIN` y `MAX` y usar __operaciones aritméticas__:

In [ ]:
%sql DROP TABLE IF EXISTS compra
%sql CREATE TABLE compra(producto VARCHAR(20), fecha VARCHAR(20), precio INT, cantidad INT)
%sql INSERT INTO compra VALUES('Leche', '01-02-2019', 890, 4)
%sql INSERT INTO compra VALUES('Chocapic', '01-02-2019', 1290, 2)
%sql INSERT INTO compra VALUES('Queso' , '01-02-2019', 990, 1)
%sql INSERT INTO compra VALUES('Pan' , '01-02-2019', 1190, 1)
%sql SELECT * FROM compra

### Suma

In [ ]:
%%sql
SELECT SUM(precio*cantidad)
FROM compra
WHERE producto = 'Leche'

### Contar tuplas
El comano __COUNT__ cuenta las tuplas. __OJO:__ Se cuentan los duplicados!

In [ ]:
%%sql
SELECT COUNT(*) as cuenta_total
FROM productos
WHERE año_fabricacion > 2018

La consulta anterior se comporta igual que esta: 

In [ ]:
%%sql
SELECT COUNT(fabricante) as cuenta_total
FROM productos
WHERE año_fabricacion > 2018

### GROUP BY
Contaremos los prouctos de cada fabricante, utilizando el operador __GROUP BY__

In [ ]:
%%sql 
SELECT fabricante, COUNT(fabricante)
FROM productos
WHERE año_fabricacion > 2018
GROUP BY fabricante

Esta consulta:
    - Computa los resultados según el FROM y el WHERE 
    - Agrupa los resultados según los atributos del GROUP BY 
    - Para cada grupo se aplica inependientemente la agregación 
Otro ejemplo:

In [ ]:
%sql DROP TABLE IF EXISTS compra
%sql CREATE TABLE compra(producto VARCHAR(20), fecha TEXT, precio INT, cantidad INT)
%sql INSERT INTO compra VALUES('tomates', '2018-02-07', 100, 6)
%sql INSERT INTO compra VALUES('tomates', '2018-07-06', 150, 4)
%sql INSERT INTO compra VALUES('zapallos', '2018-02-08', 800, 1)
%sql INSERT INTO compra VALUES('zapallos', '2018-07-09', 1000, 2)
%sql INSERT INTO compra VALUES('zapallos', '2018-01-01', 600, 3)
%sql SELECT * FROM compra

In [ ]:
%%sql
SELECT producto, SUM(precio*cantidad) AS ventaTotal
FROM compra 
WHERE fecha > '2018-01-10'
GROUP BY producto

¿Cómo funciona esto?. En primer lugar, se ejecuta el from y el where:

<table>
  <tr>
    <th>Producto</th>
    <th>Fecha</th> 
    <th>Precio</th>
    <th>Cantidad</th>
  </tr>
  <tr>
    <td>Tomates</td>
    <td>07/02</td> 
    <td>100</td>
    <td>6</td>
  </tr>
  <tr>
    <td>Tomates</td>
    <td>06/07</td> 
    <td>150</td>
    <td>4</td>
  </tr>
  <tr>
    <td>Zapallo</td>
    <td>08/02</td> 
    <td>800</td>
    <td>1</td>
  </tr>
  <tr>
    <td>Zapallo</td>
    <td>09/07</td> 
    <td>1000</td>
    <td>2</td>
  </tr>
</table>

Luego, se agrupa según el group by:

<table>
  <tr>
    <th>Producto</th>
    <th>Fecha</th> 
    <th>Precio</th>
    <th>Cantidad</th>
  </tr>
  <tr>
    <td rowspan="2">Tomates</td>
    <td>07/02</td> 
    <td>100</td>
    <td>6</td>
  </tr>
  <tr>
    <td>06/07</td> 
    <td>150</td>
    <td>4</td>
  </tr>
  <tr>
    <td rowspan="2">Zapallo</td>
    <td>08/02</td> 
    <td>800</td>
    <td>1</td>
  </tr>
  <tr>
    <td>09/07</td> 
    <td>1000</td>
    <td>2</td>
  </tr>
</table>

Finalmente, se agrega por grupo y se ejecuta la proyección:

<table>
  <tr>
    <th>Producto</th>
    <th>VentaTotal</th>
  </tr>
  <tr>
    <td>Tomates</td>
    <td>1200</td>
  </tr>
  <tr>
    <td>Zapallo</td>
    <td>2800</td>
  </tr>
</table>

### HAVING

Consideremos ahora la misma consulta, pero sólo considerando aquellos productos que se vendieron más de 100 veces (Para mostrar el efecto de este operadore, agregaremos un par de filas a la tabla)

In [ ]:
%sql INSERT INTO compra VALUES('zanahorias', '2018-07-06', 150, 125)
%sql INSERT INTO compra VALUES('zanahorias', '2018-02-08', 800, 50)

In [ ]:
%sql SELECT * FROM compra

In [ ]:
%%sql
SELECT producto, SUM(precio*cantidad) AS ventaTotal, sum(cantidad)
FROM compra 
WHERE fecha > '2018-01-10'
GROUP BY producto
HAVING sum(cantidad) > 100

¿Por qué usamos `HAVING` y no lo incluimos en el `WHERE`?

### Consultas con agregación

```SQL
SELECT <S>
FROM R_1, ..., R_N
WHERE <Condición 1>
GROUP BY a_1, ..., a_k
HAVING <Condición 2>
```

- $S$ puede contener atributos $a_1, ..., a_k$ y/o agregados, pero ningún otro atributo (¿Por qué?)
- _Condición 1_ es una condición que usa atributos $R_1, ..., R_n$
- _Condición 2_ es una condición de agregación de los atributos $R_1, ..., R_n$

#### ¿Cómo se evalúan este tipo de consultas?

1. Se computa el `FROM` - `WHERE` de $R_1, ..., R_n$
2. Se agrupa la tabla por los atributos de $a_1, ..., a_k$
3. Se computan los agregados de la _Condición 2_ y se mantienen los grupos que la satisfacen
4. Se computan los agregados de $S$ y se entrega el resultado.

Armemos un ejemplo:

In [ ]:
%sql DROP TABLE IF EXISTS Autor
%sql DROP TABLE IF EXISTS Documento
%sql DROP TABLE IF EXISTS Escribe
%sql DROP TABLE IF EXISTS Menciona

%sql CREATE TABLE Autor(login INT, nombre VARCHAR(20))
%sql CREATE TABLE Documento(id_doc VARCHAR(30), titulo VARCHAR(20))
%sql CREATE TABLE Escribe(login INT, id_doc VARCHAR(30))
%sql CREATE TABLE Menciona(id_doc VARCHAR(30), palabra VARCHAR(20))

%sql INSERT INTO Autor VALUES(0, 'Marcela Paz')
%sql INSERT INTO Autor VALUES(1, 'Pablo Neruda')
%sql INSERT INTO Autor VALUES(2, 'Gabriela Mistral')
%sql INSERT INTO Documento VALUES(0, 'Papelucho')
%sql INSERT INTO Documento VALUES(1, 'Papelucho en la clínica')
%sql INSERT INTO Documento VALUES(2, 'Papelucho detective')
%sql INSERT INTO Documento VALUES(3, 'Papelucho perdido')
%sql INSERT INTO Escribe VALUES(0,0)
%sql INSERT INTO Escribe VALUES(0,1)
%sql INSERT INTO Escribe VALUES(0,2)
%sql INSERT INTO Escribe VALUES(0,3)
%sql INSERT INTO Documento VALUES(4, 'Cien Sonetos de Amor')
%sql INSERT INTO Documento VALUES(5, 'Confieso que he vivido')
%sql INSERT INTO Escribe VALUES(1,4)
%sql INSERT INTO Escribe VALUES(1,5)
%sql INSERT INTO Documento VALUES(6, 'Locas Mujeres')
%sql INSERT INTO Documento VALUES(7, 'Poemas')
%sql INSERT INTO Documento VALUES(8, 'Desolación')
%sql INSERT INTO Escribe VALUES(2,6)
%sql INSERT INTO Escribe VALUES(2,7)
%sql INSERT INTO Escribe VALUES(2,8)


Encontremos a todos los autores que escribieron al menos 3 documentos:

In [ ]:
%%sql
SELECT DISTINCT *
FROM (SELECT Autor.login, COUNT(*) as countAutor
      FROM Escribe, Autor
      WHERE Escribe.login = Autor.login
      GROUP BY Autor.login) AS Foo
WHERE countAutor >= 3

¡La respuesta puede mejorar! Estamos haciendo una anidación innecesaria.

__Obs:__ El uso del alias (en este caso: Foo) en el FROM es necesario para el uso de subconsultas

Veamos una versión más elegante:

In [ ]:
%%sql
SELECT Autor.nombre
FROM Autor, Escribe
WHERE Autor.login = Escribe.login
GROUP BY Autor.nombre
HAVING COUNT(Escribe.id_doc) >= 3

__NOTA:__ No necesitamos __DISTINCT__ gracias al GROUP BY :D 

Otro ejemplo: Encontrar a todos los autores con un vocabulario de más de 10000 palabras. 

```SQL
SELECT Autor.nombre
FROM Autor, Escribe, Menciona
WHERE Autor.login = Escribe.login AND Escribe.id_doc = Menciona.id_doc
GROUP BY Autor.nombre
HAVING COUNT(DISTINCT Menciona.palabra ) >= 10000
```